In [1]:
# Function to extract focal channels
def extract_fnsz_channels(file_path):
    """
    Extract channel names corresponding to rows with the label 'fnsz' for each start time,
    merging start times that are 5 seconds or less apart into the earliest start time.

    Args:
        file_path (str): Path to the CSV file.

    Returns:
        dict: A dictionary where keys are start times and values are lists of channel names.
    """
    try:
        # Load CSV data, skipping lines that start with '#'
        data = pd.read_csv(file_path, comment='#')
        
        # Filter rows with the label 'fnsz'
        fnsz_data = data[data['label'] == 'fnsz']
        
        # Helper function to convert differential montage channels to detailed channel names
        def convert_channel_to_detailed(channel):
            return channel.split('-')  # Split into parts for detailed representation
        
        # Sort start times and merge them if they are within 5 seconds
        sorted_times = sorted(fnsz_data['start_time'].unique())
        merged_start_times = []
        current_group = [sorted_times[0]]

        for i in range(1, len(sorted_times)):
            if sorted_times[i] - current_group[-1] <= 5:
                current_group.append(sorted_times[i])
            else:
                merged_start_times.append(current_group)
                current_group = [sorted_times[i]]
        merged_start_times.append(current_group)  # Add the last group
        
        # Create a mapping from each original start time to its merged start time
        start_time_mapping = {
            time: group[0] for group in merged_start_times for time in group
        }
        
        # Map each start time to the merged time
        fnsz_data['merged_start_time'] = fnsz_data['start_time'].map(start_time_mapping)
        
        # Group by 'merged_start_time' and extract unique, detailed channels for each group
        focal_zone_channels = (
            fnsz_data.groupby('merged_start_time')['channel']
            .apply(lambda channels: list(
                set(
                    detailed_channel
                    for ch in channels.unique()
                    for detailed_channel in convert_channel_to_detailed(ch)
                ) & set(electrodes)  # Filter channels based on the predefined 'electrodes' set
            ))
            .to_dict()
        )
        
        return focal_zone_channels

    except Exception as e:
        print(f"Error processing file '{file_path}': {e}")
        return {}